
# 형태소 분석 기반 토큰화의 문제
- 형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
- 그래서 발생 할 수있는 잠재적 문제점
    - 어휘사전을 크게 만든다.
        - 같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다.
        - ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다.
    - OOV(Out Of Vocab)에 대응하기 어렵게 만든다.
        - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.


## 어휘 사전(Vocabulary)과 Out Of Vocabulary (OOV)

- 어휘사전(Vocab)은 토크나이저(Tokenizer)가 사용하는 모든 토큰의 집합이며,**각 토큰**을 고유한 **정수 ID**에 매핑한 사전이다. 토크나이저가 텍스트를 토큰 ID 시퀀스로 변환할 때 기준으로 사용된다. 

   - 매핑된 정수는 모델에 입력되는 텍스트 데이터를 숫자 형식으로 변환해 모델이 처리할 수 있도록 돕는다.
   - 예
        ```bash
        {
            "자연어": 0,
            "처리": 1,
            "는": 2,
            "재미있다": 3,
            "공부": 4,
        }
        ```
- **Out Of Vocabulary (OOV)**
   - 어휘 사전(Vocab): 코퍼스를 구성하는 모든 토큰의 집합.
   - **OOV**란 어휘 사전에 포함되지 않은 토큰을 의미하며, 모델이 해당 토큰을 처리할 수 없기 때문에 일반적으로 특별한 토큰(예: `[UNK]`)으로 대체되거나 다른 방식으로 처리된다.

# Subword Tokenization(하위 단어 토큰화)

## 정의

- Subword Tokenization은 단어를 더 작은 단위(subword)로 나누어 텍스트를 토큰화하는 방식이다.  
    - subword는 하나의 단어를 구성하는 단어들을 말한다.(coworker: co, work, er)
- 주로 자주 등장하는 단어의 일부를 공통된 토큰으로 만들고, 희귀하거나 복합적인 단어는 작은 조각(subword)으로 나누어 처리한다.
- 단어 자체를 그대로 사용하기보다는 단어의 일부를 나누어 처리함으로써 새로운 단어나 미등록 단어(Out-of-Vocabulary) 문제를 줄일 수 있다.

## 장점

1. **미등록 단어 처리 가능**  
   -  새로운 단어(신조어, 속어, 고유어등)가 등장해도 미리 정의된 subword를 조합해서 표현할 수 있어 OOV 문제를 줄일 수 있다.  

2. **어휘 크기 축소**  
   - 같은 subword를 여러 단어에서 공유함으로써, 완전한 단어를 사용하는 경우보다 어휘집의 크기를 작게 유지할 수 있다.


## 종류

1. **Byte-Pair Encoding (BPE)**  
   - 자주 등장하는 문자 쌍을 반복적으로 병합해 서브워드를 생성하는 방식.
   - OpenAI의 GPT 모델에 사용된 토크나이저이다.

2. **Unigram**  
   - 빈도기반 확률모델에 따라 subword 단위를 선택하는 방식이다.  
   - BPE보다 유연하여 더 다양한 분할 결과를 얻을 수 있다.

3. **WordPiece**  
   - BPE와 유사하지만, 빈도수가 아니라, 가능성이 높은 조합(합쳐질 가능성이 높은 subword)에 기반해 subword들을 찾는다.
   - Google의 BERT 모델에 사용된 토크나이저이다.

# Byte Pair Encoding 방식

- 원래 Text data 압축을 위해 만들어진 방법으로 text 에서 자주 같이 나오는 두 글짜 쌍을 합쳐서 하나의 부호(기호,글자)로 만들어 나가면서 글자 수를 줄이는 알고리즘이다. 
- 연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예
- 원문: abracadabra
1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 맣은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식
BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)
1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다. (Pre Tokenization)
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다.  위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

- 위와 같이 어휘 사전이 만들어 지면 원래 어휘서전에 없던 것들에 대한 처리를 할 수있다.
    - ex)
        - 'newer' :=> 'new', 'e', 'r', 
        - 'lowest' :=> 'low', 'est'
        - 'wider' :=> 'wid', 'e', 'r'

# WordPiece tokenizer

- Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
- 두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)} 
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
- 가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
- 'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
- 나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.

# Unigram 방식
- 빈도 기반 확률 모델을 사용하여 효율적으로 서브워드를 선택하고, 불필요한 서브워드를 제거해 최적의 어휘 크기를 찾는 알고리즘


- **초기 어휘 집합 구성**
    - 대상 text에 모든 단어와 그 서브스트링을 포함한 어휘 집합을 생성한다. 이 어휘 집합은 나올 수있는 모든 subword들을 다 모아놓은 것이다. 
    - 예를 들어 "hug" 단어의  ["h", "u", "g", "hu", "ug", "hug"]  substring을 만든다. 이들이 subword 후보가 된다.
- **각 Subword의 빈도수 기반 확률 계산**
    -  $\cfrac{subword가\;나타난\;횟수}{전체\;빈도수}$ 로 각 subword들의 나타난 확률을 계산한다.
- **가능한 분할에 대한 확률 계산**
    - 단어를 여러 서브워드로 분할할 수 있는 경우, 각 분할에 대한 전체 확률을 계산한다.
    - 확률 계산은 $ P(subword1)\;\times \; P(subword2)\;\times\; ..$ 으로 계산한다.
    - 예를 들어 "hug" 를 분할 한다고 했을 때
        1. \["h", "u", "g"\]: $ P(h) \times P(u) \times P(g) $
        2. \["hu", "g"\]: $ P(pu) \times P(g) $

   - 각각의 확률을 계산한 후, **가장 높은 확률**을 가진 분할을 선택한다.
     - 위 예에서 만약 1의 확률이 0.01 이고 2의 확률이 0.00001 이라면 첫번째 분할이 선택된다.

- **서브워드 제거**
    - 위의 훈련 과정에서 불필요한 서브워드를 제거하면서 최적의 어휘 집합을 찾아간다. 
    - 제거 대상은 빈도수가 낮거나 조합에 크게 영향을 주지 않은 subword들이다.

> ### korpora 말뭉치
> - 다양한 한글 데이터셋을 제공하는 패키지
> - `pip install korpora`

# Hugging Face tokenizers 패키지 사용해 토큰화 수행

## 주요 라이브러리 
- [tokenizers](https://huggingface.co/docs/tokenizers/index)
    - huggingface에서 개발한 tokenizer 라이브러리로 BPE, WordPiece, Unigram 알고리즘을 지원한다. 
    - 설치: `pip install tokenizers`
- [Sentencepiece](https://github.com/google/sentencepiece)
    - 구글에서 개발한 subword tokenizer 라이브러리로 BPE, Unigram 방식 지원.
    - 설치: `pip install sentencepiece`

## Hugging Face tokenizers 패키지이용
- Hugging face는 
  - Subword방식의 토크나이저를 생성할 수 있는 알고리즘을 제공한다. `tokenizers` library를 이용해 사용할 수 있다.
  - GPT, BERT등 다양한 LLM 모델에서 사용된 Pretrained Tokenizer들을 제공한다. `transformers` Library를 이용해 사용할 수 있다.
- 설치: `pip install tokenizers`
- Tokenizer 생성
    - 토큰화 알고리즘을 지정해 instance 생성.
- Trainer 생성
    - 학습 파라미터를 설정해서 instance 생성
- Tokenizer 학습
    - train() 메소드: 학습 text 파일 경로를 지정해서 학습
    - train_from_iterator() 메소드: 학습할 string들을 iterator를 통해 제공.
- https://github.com/huggingface/tokenizers

In [ ]:
# 테스트 문장
sports_txt = "프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다."
petition_txt = "이 글을 쓴 이유는 다름아닌 '전안법'시행 반대를 주장하기 위해서입니다. 먼저, '전안법'은 전기용품 및 생활용품을 판매하는 업체에서 KC인증마크를 의무적으로 받는 것입니다."
comment_txt = "멋진 식사를 즐기기에 좋은 장소 - 채식 메뉴가 정말 훌륭했습니다. 당근 케이크는 아마도 내가 먹어본 디저트 중 최고였을 거예요."